In [ ]:
#Import libraries
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import KFold
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
import random
from numpy import linalg as LA

In [ ]:
def load_model(model_path):
    model = Sequential()
    model.add(Flatten(input_shape=(28, 28)))
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
    model.add(Dense(10, activation='softmax', kernel_regularizer=tf.keras.regularizers.l2(0.001)))

    return model

In [ ]:
# Load train and test dataset
def load_dataset():
    # load dataset
    (trainX, trainY), (testX, testY) = mnist.load_data()
    # reshape dataset to have a single channel
    trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
    testX = testX.reshape((testX.shape[0], 28, 28, 1))
    # one hot encode target values
    trainY = to_categorical(trainY)
    testY = to_categorical(testY)

    random_indices = np.random.choice(len(trainX), 10016, replace=False)

    # Select the subset of data and labels
    subset_X = trainX[random_indices]
    subset_Y = trainY[random_indices]
    return subset_X, subset_Y, testX, testY

In [ ]:
# scale pixels
def prep_pixels(train, test):
 # convert from integers to floats
 train_norm = train.astype('float32')
 test_norm = test.astype('float32')
 # normalize to range 0-1
 train_norm = train_norm / 255.0
 test_norm = test_norm / 255.0
 # return normalized images
 return train_norm, test_norm

In [ ]:
trainX = []
trainY = []
for i in range (0,6):
  temp1, temp2, testX, testY = load_dataset()
  temp1, testX = prep_pixels(temp1, testX)
  trainX.append(temp1)
  trainY.append(temp2)

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
class ClientProx:
    def __init__(self, trainX, trainY, testX,
                 testY, batchSize, model, regRate, gamma,
                 loss, metrics, lr, optim=tf.keras.optimizers.legacy.SGD()):

        self.model = model
        self.trainX = trainX
        self.trainY = trainY
        self.testX = testX
        self.testY = testY
        self.batchSize = batchSize
        self.lr = float(lr)
        self.losses = loss
        self.metrics = metrics
        self.optim = optim
        self.optim.learning_rate = self.lr
        self.regRate = regRate
        self.gamma = gamma

    def train(self, Global):
        print('started')
        self.model.compile(optimizer=self.optim,
                           loss=self.losses, metrics=self.metrics)
        self.model.set_weights(Global)
        count = 0
        while(self.condition(Global) and count  < 1000):
          count = count + 1
          self.model.fit(self.trainX, self.trainY, verbose=0, batch_size = self.batchSize)

        weights = self.model.get_weights()
        delta_weights = [(w - wt) for w, wt in zip(weights,Global)]

        return delta_weights, len(self.trainX)

    def condition(self,Global):
        x = 0
        weights = self.model.trainable_weights
        h = self.calculate_loss(weights, Global)
        globalH = self.calculate_loss(Global, Global)

        return bool(h > self.gamma * globalH)

    def calculate_loss(self, W , Wt):
        dummy_model = load_model('')
        dummy_model.set_weights(W)
        weights = dummy_model.trainable_weights
        with tf.GradientTape() as tape:
          predictions = dummy_model(self.trainX)
          loss = self.losses(self.trainY, predictions)

        grad_local_model_loss = tape.gradient(loss, weights)
        grad_regularization_term = [self.regRate * (w - wt) for w, wt in zip(weights, Wt)]
        h = [w + reg for w, reg in zip(grad_local_model_loss, grad_regularization_term)]
        h = [LA.norm(w) for w in h]
        h = np.array(h)
        h = LA.norm(h)
        return h

In [ ]:
class Server:
  def __init__(self, global_lr, clients_num):
    self.global_model = load_model(model_path = " ")
    self.global_model.compile(
    optimizer=tf.keras.optimizers.SGD(0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.CategoricalAccuracy()],
    )
    self.clients_num = clients_num

  def initiate(self):
    self.clients = []
    optim = tf.keras.optimizers.legacy.SGD()
    for i in range(0,self.clients_num):
      self.clients.append(ClientProx(trainX[i], trainY[i], testX, testY, 32
                                         ,load_model(""), 0.1 , 0.4
                                         ,tf.keras.losses.CategoricalCrossentropy(from_logits=True)
                                         ,[tf.keras.metrics.CategoricalAccuracy()]
                                         , 0.01,optim))

  def train(self, rounds):
      weights = self.global_model.get_weights()
      for j in range(0,rounds):
        for client in self.clients:
             [delta_weights , num_val] = client.train(self.global_model.get_weights())
             for i in range(0 , len(weights)):
                  weights[i] = weights[i] + (delta_weights[i] * (1.0  /float(self.clients_num)))
        self.global_model.set_weights(weights)
        print("Global:",self.global_model.evaluate(testX, testY),"\n")

In [ ]:
server = Server(1, 6)

In [ ]:
server.initiate()

In [ ]:
server.train(2)

started
started
started
started
started
started
313/313 [==============================] - 1s 2ms/step - loss: 1.1206 - categorical_accuracy: 0.8191
Global: [1.1206109523773193, 0.819100022315979] 

started
started
started
started
started
started
313/313 [==============================] - 1s 2ms/step - loss: 0.6174 - categorical_accuracy: 0.9014
Global: [0.6174463629722595, 0.9014000296592712] 



In [ ]:
  #bare bones SGD with scaffold updates using gradient tape (Similar to pytorch)

  ##loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
  ##optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
  ##train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
  ##val_acc_metric = tf.keras.metrics.CategoricalAccuracy()
  ##for i in range(0,len(self.trainX)):
  ## with tf.GradientTape() as tape:
  ##    logits = self.model(self.trainX[i], training=True)
  ##    loss_value = loss_fn(self.trainY[i], logits)
  ## grads = tape.gradient(loss_value, self.model.trainable_weights)
  ## for i in range (0 , len(Global)):
  ##    grads[i] += 1e-3 * (C[i] - self.c[i])
  ## optimizer.apply_gradients(zip(grads, self.model.trainable_weights))



      ##val_logits = self.model(self.testX, training=False)
      # Update val metrics
  ##val_acc_metric.update_state(self.testY, val_logits)
  ##val_acc = val_acc_metric.result()
  ##print("Client : Validation acc: %.4f" % (float(val_acc),))
  ##val_acc_metric.reset_states()
  ##results = self.model.evaluate(self.testX , self.testY, batch_size=128)